In [2]:
import torchio as tio
from torchio.data.io import sitk_to_nib
import SimpleITK as sitk
import napari
import numpy as np
import nibabel as nib

from utils.image import read_im_gt, read_reorient_nifti

In [3]:
img_path = '/home/t722s/Desktop/Datasets/Dataset350_AbdomenAtlasJHU_2img/imagesTr/BDMAP_00000001_0000.nii.gz'
gt_path = '/home/t722s/Desktop/Datasets/Dataset350_AbdomenAtlasJHU_2img/labelsTr/BDMAP_00000001.nii.gz'
class_label = 3

gt_unprocessed = nib.load(gt_path).get_fdata()
gt_unprocessed = np.where(gt_unprocessed == class_label, 1, 0)

img, gt = read_im_gt(img_path, gt_path, class_label, RAS=True)

gt_2, _ = read_reorient_nifti(gt_path, np.float32, RAS = True)
gt_2 = np.where(gt_2 == 3, 1, 0)

np.array_equal(gt, gt_2)


True